# MindQuantum 实战与作业


## 量子模拟器

MindQuantum 中量子模拟器维护着一个量子态，通过在量子模拟器上作用一些量子门或者量子线路，我们可以改变这些量子态。

In [ ]:
#!pip insatll mindquantum==0.7.0

In [1]:
from mindquantum.simulator import Simulator

sim = Simulator('projectq', 1)  # 第一个参数为模拟器的名字，第二个参数为模拟器的比特数
sim

projectq simulator with 1 qubit (little endian).
Current quantum state:
1¦0⟩

In [2]:
state = sim.get_qs()
print(state)

[1.+0.j 0.+0.j]


由上可知，量子模拟器的初始状态为 $\left|0\right>$ 态，且通过接口 `get_qs` 我们可以获取到模拟器中的量子态。下面我们学习如何将模拟器的初态设置成 $\left|1\right>$ 态。

In [3]:
import numpy as np

sim = Simulator('projectq', 1)
state1 = np.array([0.0, 1.0])   #生成（0,1）数据的态 |1> 态
state_before = sim.get_qs()
sim.set_qs(state1)
state_after = sim.get_qs()
print("Initial state: ", state_before)
print("Last state: ", state_after)

Initial state:  [1.+0.j 0.+0.j]
Last state:  [0.+0.j 1.+0.j]


## 习题一

请实例化一个 `'projectq'` 的单比特模拟器，并将模拟器状态设置为:

$$\left| \psi \right> = \frac{1}{\sqrt{2}}\left( \left|0\right> + \left|1\right> \right)$$

In [5]:
# 习题一解答：

import numpy as np
from mindquantum.simulator import Simulator

sim = Simulator('projectq', 1)
psi = np.array([np.sqrt(2),np.sqrt(2)])
sim.set_qs(psi)
state_psi = sim.get_qs()
print(state_psi)

[0.70710678+0.j 0.70710678+0.j]


## 量子门

MindQuantum 中以及预定义了一些基本量子门，如 `X`、`Y`、`Z`、`RX`、`RY`、`RZ`等等。量子门是构成量子算法的基本组成单元。

我们以`X`门为例，介绍量子门的使用。

In [6]:
from mindquantum.core.gates import X

print(X.on(0))

X(0)


如上，我们便生成了一个作用在`0`比特上的`X`门。我们知道，在量子计算中，量子门是可以被量子比特控制的。在MindQuantum中，我们可以很简单的生成受控量子门，只需给`on`接口的第二个参数传入量子比特序号即可，如：

In [7]:
cnot = X.on(0, 1)
toffoli = X.on(0, [1, 2])
print("cnot gate: ", cnot)
print("toffoli gate: ", toffoli)

cnot gate:  X(0 <-: 1)
toffoli gate:  X(0 <-: 1 2)


如上我们便利用`X`门生成了`CNOT`门和`Toffoli`门。此外，我们还可以获取量子门的矩阵，如

In [8]:
print(X.matrix())

[[0 1]
 [1 0]]


需要注意的是，这里我们会忽略控制比特的效果

In [9]:
print(cnot.matrix())

[[0 1]
 [1 0]]


除了上面这种矩阵元素固定的量子门，量子计算中还有含参数量子门，如旋转门`RX`，我们可以指定它的旋转角度

In [9]:
import numpy as np
from mindquantum.core.gates import RX


rx_s2 = RX(np.sqrt(2)).on(0)
print(rx_s2)
print(rx_s2.matrix())

RX(√2|0)
[[0.7602446+0.j         0.       -0.64963694j]
 [0.       -0.64963694j 0.7602446+0.j        ]]


如上，我们得到了作用在0比特上绕`X`轴旋转 $\sqrt{2}$ 的量子门。

## 习题二

利用MindQuantum证明，`X` 门是绕着 `X` 轴转动 $\pi$ 的`RX`门（忽略全局相位）。

In [12]:
# 习题二解答

import numpy as np
from mindquantum.core.gates import X, RX

RX_pi = RX(np.pi).on(0)
print(RX_pi.matrix())
print(X.matrix())

[[0.+0.j 0.-1.j]
 [0.-1.j 0.+0.j]]
[[0 1]
 [1 0]]


## 量子线路

量子线路是量子门的集合，我们可以将量子门添加到量子线路中来构成想要的量子算法。

In [11]:
from mindquantum.core.gates import H
from mindquantum.core.circuit import Circuit

circ = Circuit()
circ += H.on(0)
circ += H.on(1)
print(circ)

q0: ──H──

q1: ──H──


如上，我们便构建好了一个最简单的量子线路，这个量子线路中有两个Hadmard门，分别作用在0号和1号比特上，该线路可以制备一个两比特的均匀叠加态：

$$\left|\psi\right> = \frac{1}{2} \left(\left|00\right> + \left|01\right> + \left|10\right> + \left|11\right>\right)$$

MindQuantum中还预定义了许多量子线路，例如用于进行量子傅里叶变化的线路：

In [16]:
from mindquantum.algorithm.library import qft

qft_circ = qft(range(2))
qft_circ.svg()

## 习题三

请利用MindQuantum生成一个用于制备纠缠态的量子线路，该纠缠态为：

$$\left|\psi\right> = \frac{1}{\sqrt{2}} \left( \left|01\right> + \left|10\right> \right)$$

In [16]:
# 习题三解答

from mindquantum.core.gates import X, H
from mindquantum.core.circuit import Circuit

circ = Circuit()
circ += H.on(0)
circ += X.on(1)
circ += X.on(1,0)
print(circ)

sim = Simulator('projectq', circ.n_qubits)
sim.apply_circuit(circ)
sim

q0: ──H────●──
           │
q1: ──X────X──


projectq simulator with 2 qubits (little endian).
Current quantum state:
√2/2¦01⟩
√2/2¦10⟩

## 量子模拟器的演化

有了上面的介绍，我们便可以将量子线路作用到模拟器上，来演化模拟器中的量子态。例如，我们将证明上一小节中的量子线路确实能够制备两比特的均匀叠加态。

In [17]:
from mindquantum.core.circuit import Circuit
from mindquantum.core.gates import H

circ = Circuit()
circ += H.on(0)
circ += H.on(1)

sim = Simulator('projectq', circ.n_qubits)   #circ.n_qubits可直接获得qubits数量

sim.apply_circuit(circ)
print(sim)

projectq simulator with 2 qubits (little endian).
Current quantum state:
1/2¦00⟩
1/2¦01⟩
1/2¦10⟩
1/2¦11⟩


在上例中，我们通过调用`apply_circuit`接口，实现了将量子线路作用到量子模拟器上的任务。

## 习题四

请验证习题三中的线路确实能够生成给定的纠缠态。

In [17]:
# 习题四解答
from mindquantum.core.gates import X, H
from mindquantum.core.circuit import Circuit

circ = Circuit()
circ += H.on(0)
circ += X.on(1)
circ += X.on(1,0)
print(circ)

sim = Simulator('projectq', circ.n_qubits)
sim.apply_circuit(circ)
sim


q0: ──H────●──
           │
q1: ──X────X──


projectq simulator with 2 qubits (little endian).
Current quantum state:
√2/2¦01⟩
√2/2¦10⟩